In [1]:
import pvlib
import json
from collections import Counter

In [2]:
modules = pvlib.pvsystem.retrieve_sam(
            "CECMod", path="CEC Modules.csv"
        )

In [3]:
names_split = [name.split("_") for name in modules.keys()]

In [4]:
occurences = Counter([part for parts_list in names_split for part in parts_list])

In [25]:
panel_database = {}
for i, name_split in enumerate(names_split):
    name_split_no_empty = [part for part in name_split if part != ""]
    manufacturer = name_split_no_empty[0]
    series = ""
    model = name_split_no_empty[-1]
    
    left_over_parts =  name_split_no_empty[1:-1]
    while len(left_over_parts) > 0:
        if len(left_over_parts) == 1 and series == "": # series
            series = left_over_parts.pop(0)
        elif occurences[left_over_parts[0]] > 1 and not any([c.isdigit() for c in left_over_parts[0]]): #manufacturer
            manufacturer += " " + left_over_parts.pop(0)
        elif occurences[left_over_parts[-1]] == 1: # model
            model = left_over_parts.pop(-1) + " " + model
        else: # series
            if series != "":
                series += " " + left_over_parts.pop(0)
            else:
                series = left_over_parts.pop(0)
    if manufacturer not in panel_database:
        panel_database[manufacturer] = {}
    if series not in panel_database[manufacturer]:
        panel_database[manufacturer][series] = {}
    panel_database[manufacturer][series][model] = modules["_".join(name_split)].to_dict()

In [26]:
panel_database["SunPower SPR"]["X22"]["370"]

{'Technology': 'Mono-c-Si',
 'Bifacial': 0,
 'STC': 370.116,
 'PTC': 343.9,
 'A_c': 1.63,
 'Length': nan,
 'Width': nan,
 'N_s': 96,
 'I_sc_ref': 6.6,
 'V_oc_ref': 70.1,
 'I_mp_ref': 6.21,
 'V_mp_ref': 59.6,
 'alpha_sc': 0.002297,
 'beta_oc': -0.199925,
 'T_NOCT': 46.84,
 'a_ref': 2.630367,
 'I_L_ref': 6.604403,
 'I_o_ref': 1.726423e-11,
 'R_s': 0.360284,
 'R_sh_ref': 539.995911,
 'Adjust': 3.030361,
 'gamma_r': -0.3509,
 'BIPV': 'N',
 'Version': 'SAM 2018.11.11 r2',
 'Date': '1/3/2019'}

In [27]:
with open("G:/projects/solar-power-estimator/data/panel_database.json", "w") as f:
    json.dump(panel_database, f)